In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
trainX = np.array(((0,0),(0,1),(1,0),(1,1)))
trainY = np.array((0,1,1,1))

In [405]:
print(trainX.shape)
print(trainY.shape)

(4, 2)
(4,)


In [406]:
trainX = np.transpose(trainX)

trainY = trainY.reshape(1,trainY.shape[0])

print(trainX.shape)
print(trainY.shape)

(2, 4)
(1, 4)


In [407]:
layers = np.array((trainX.shape[0],10,7,5,3,trainY.shape[0]))

print("Model has " +str(layers.shape[0])+ " Layers")
for i in range(1,layers.shape[0]):
  print("Layer " +str(i)+ " has " +str(layers[i])+ " Nodes")

Model has 6 Layers
Layer 1 has 10 Nodes
Layer 2 has 7 Nodes
Layer 3 has 5 Nodes
Layer 4 has 3 Nodes
Layer 5 has 1 Nodes


In [0]:
def initWeights(layers):
  #Random Initialization

  L = len(layers)
  param={}

  for i in range(1,L):
    param["W"+str(i)] = np.random.randn(layers[i], layers[i-1]) *0.75
    param["b"+str(i)] = np.zeros((layers[i],1))

  return param

In [0]:
def forwardActivation(A_prev, W, b, activation):

  Z = np.dot(W,A_prev) + b

  if activation == "relu":
    A = np.maximum(Z,0)

  if activation == "sigmoid":
    A = 1/(1+np.exp(-Z))

  l_cache = (A_prev, W, b)
  a_cache = (A, Z)
  cache = (l_cache, a_cache)

  return A, cache

In [0]:
def forwardProp(X, param):

  L = len(param)//2
  caches = []
  A = X

  for i in range(1,L):
    A_prev = A
    W = param["W"+str(i)]
    b = param["b"+str(i)]
    A, cache = forwardActivation(A_prev, W, b, activation="relu")
    caches.append(cache)

  W = param["W"+str(L)]
  b = param["b"+str(L)]
  A_last_layer, cache = forwardActivation(A, W, b, activation="sigmoid")
  caches.append(cache)

  return A_last_layer, caches

In [0]:
def computeCost(m, A, Y):

  loss = 1/-m * (Y*np.log(A)) + ((1-Y)*np.log(1-A))

  return loss

In [0]:
def computeL2Cost(m, lambd, param):

  L = len(param)//2

  for i in range(1,L):
    W = np.sum(np.square(param["W" +str(i)]))
    #b is very small so almost negligible

  W = W + np.sum(np.square(param["W" +str(L)]))

  l2_cost = ((lambd*W)/(2*m))

  return l2_cost

In [0]:
def backwardActivation(m, dA, Y, cache, lambd, activation):

  l_cache, a_cache = cache

  A_prev, W, b = l_cache
  A, Z = a_cache

  if activation == "sigmoid":
    dZ = dA - Y

  if activation == "relu":
    dZ = np.multiply(dA, np.int64(A > 0))

  dW = (np.dot(dZ,A_prev.T)/m) + ((lambd*W)/m)
  db = np.sum(dZ, axis=1, keepdims=True)
  dA_prev = np.dot(W.T,dZ)

  return dA_prev, dW, db

In [0]:
def backwardProp(m, A, Y, lambd, caches):

  grads= {}
  L = len(caches)
  m = A_last_layer.shape[1]

  dA_current = -(np.divide(Y,A) - np.divide((1-Y),(1-A)))
  cache_last_layer = caches[-1]

  dA_prev, dW, db = backwardActivation(m, dA_current, Y, cache_last_layer, lambd, activation="sigmoid")
  grads["dA" +str(L-1)] = dA_prev
  grads["dW" +str(L)] = dW
  grads["db" +str(L)] = db

  for i in reversed(range(L-1)):
    dA_current = grads["dA" +str(i+1)]
    cache_current_layer = cache[i]

    dA_prev, dW, db = backwardActivation(m, dA_current, Y, cache_current_layer, lambd, activation="relu")
    grads["dA" +str(i)] = dA_prev
    grads["dW" +str(i+1)] = dW
    grads["db" +str(i+1)] = db 

  return grads

In [0]:
def optimize(lr, param, grads):

  L = len(param)//2

  for i in range(1,L):
    param["W" +str(i)] = param["W" +str(i)] - lr*grads["dW" +str(i)]
    param["b" +str(i)] = param["b" +str(i)] - lr*grads["db" +str(i)]

  return param

In [0]:
lr = float(input("Enter the learning rate: "))
iter = int(input("Enter the number of iterations: "))
lambd = float(input("Enter the value of Lambda: "))
m = trainX.shape[1]
cost_plot = []

param = initWeights(layers)
#param contains W and B for each layer


for i in range(1,iter):
  A_last_layer, cache = forwardProp(trainX, param)
  #cache format = [Layer][l_cache/a_cache][Parameter]

  cost = computeCost(m, A_last_layer, trainY)
  l2_cost = computeL2Cost(m, lambd, param)
  #L2 Regularization
  cost = np.sum(cost + l2_cost)
  if i%5 == 0:
    print(cost)
    #print(A_last_layer)
    cost_plot.append(cost)

  grads = backwardProp(m, A_last_layer, trainY, lambd, cache)
  #grads contains dW and db for each layer

  param = optimize(lr, param, grads)
  #Gradient Descent Optimizer

plt.plot(cost_plot)
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.show()

Enter the learning rate: 0.001


In [448]:
x = int(input("Enter the value of x: "))
y = int(input("Enter the value of y: "))

if (x!=0 and x!=1) or (y!=0 and y!=1):
  print("Invalid")

pred = np.array((x,y))
pred = pred.reshape((pred.shape[0],1))

result, cache = forwardProp(pred,param)

print(result)


if result>0.5:
  result = 1
else:
  result = 0

print("OR gate output is: "+str(result))

Enter the value of x: 1
Enter the value of y: 1
[[0.5]]
OR gate output is: 0
